In [ ]:
import sys
sys.path.append('..')
%load_ext autoreload
%autoreload 2
%matplotlib inline

In [ ]:
from quick_pp.objects import Project

# Load well from saved file
project = "MOCK_clastic"
project_path = rf"data\04_project\{project}.qppp"
project = Project().load(project_path)
project.get_well_names()

# Quick PP Interpretation

In [12]:
from quick_pp.lithology.sand_silt_clay import SandSiltClay
from quick_pp.porosity import neu_den_xplot_poro, density_porosity, rho_matrix, normalize_volumetric, clay_porosity
from quick_pp.qaqc import badhole_flagging, mask_outside_threshold, neu_den_xplot_hc_correction, den_correction
from quick_pp.saturation import *
from quick_pp.plotter.plotter import *
from quick_pp.permeability import *
from quick_pp.ressum import *
from quick_pp.rock_type import rock_typing, estimate_vsh_gr
from quick_pp.plotter.plotter import plotly_log
from quick_pp.utils import zone_flagging

In [ ]:
df = project.get_all_data()[['WELL_NAME', 'DEPTH', 'ZONES', 'GR', 'RT', 'NPHI', 'RHOB', 'CALI', 'BS']]
for well_name, well_data in df.groupby('WELL_NAME'):
    # Load well object
    well = project.get_well(well_name)

    # Clean up data
    well_data = well_data.dropna(subset=['NPHI', 'RHOB', 'RT', 'GR'])
    well_data = badhole_flagging(well_data)
    well_data = mask_outside_threshold(well_data, fill=True)

    # Initialize lithology model
    args = {
        'litho_model': 'ssc',
        # 'dry_clay_point': (.3, 2.7),
        'silt_line_angle': 118,
        'wet_clay_point': (0.45, 2.45),
        'sw_water_salinity': 15000,
        'sw_m': 1.85,
        'sw_n': 1.85,
        'hc_corr_angle': 50,
        'hc_buffer': 0.1,
        'ressum_cutoffs': dict(
            VSHALE=.5,
            PHIT=0,
            SWT=1
        ),
    }
    ssc_model = SandSiltClay(**args)
    vsand, vsilt, vcld, vclb = ssc_model.estimate_lithology(
        well_data['NPHI'], well_data['RHOB']
    )
    args.update(ssc_model.__dict__)
    well.update_config(args)  # Save lithology model to well

    # Implement hydrocarbon correction
    vsh_gr = estimate_vsh_gr(well_data['GR'], min_gr=0, max_gr=140)
    nphihc, rhobhc, hc_flag = neu_den_xplot_hc_correction(
        well_data['NPHI'], well_data['RHOB'], gr=well_data['GR'], vsh_gr=vsh_gr,
        dry_min1_point=args['dry_sand_point'],
        dry_clay_point=args['dry_clay_point'],
        corr_angle=args['hc_corr_angle'], buffer=args['hc_buffer']
    )
    
    # Correct density log
    rhob_corr = den_correction(nphihc, well_data['GR'], vsh_gr=vsh_gr, alpha=0.1)
    badhole_flag =  np.where(abs(well_data['RHOB'] - rhob_corr) > 0.2, 1, 0)
    rhob_corr = np.where((badhole_flag == 1) & (hc_flag == 0), rhob_corr, rhobhc)

    # Estimate lithology
    ssc_model = SandSiltClay(**args)
    vsand, vsilt, vcld, vclb = ssc_model.estimate_lithology(
        nphihc, rhob_corr
    )

    # Estimate porosity
    phit = neu_den_xplot_poro(
        nphihc, rhob_corr, model='ssc',
        dry_min1_point=args['dry_sand_point'],
        dry_silt_point=args['dry_silt_point'],
        dry_clay_point=args['dry_clay_point'],
    )

    rho_ma = rho_matrix(vsand, vsilt, vcld)
    phid = density_porosity(rhob_corr, rho_ma)
    
    # Normalize lithology
    vsand, vsilt, vcld = normalize_volumetric(vsand, vsilt, vcld, phit)

    # Calculate vclb: volume of clay bound water and phie
    clay_phit = clay_porosity(rhob_corr, args['dry_clay_point'][1])
    vclb = vcld * clay_phit
    vclw = vcld + vclb
    phie = phit - vclb
    phie = phit - vclb

    # Estimate water saturation
    temp_grad = estimate_temperature_gradient(well_data['DEPTH'], 'metric')  # Arbitrarily use MD instead of TVD
    water_salinity = args['sw_water_salinity']
    rw_salinity = estimate_rw_temperature_salinity(temp_grad, water_salinity)
    b = estimate_b_waxman_smits(temp_grad, rw_salinity)
    qv = estimate_qv(vcld, phit, cec_clay=.1)

    rw = estimate_rw_from_shale_trend(well_data['RT'], phit, m=1.2)
    m = args['sw_m']

    swt = waxman_smits_saturation(well_data['RT'], rw, phit, qv)

    # Estimate permeability
    constant = vclb**1.75
    Swirr = constant / phit
    perm = choo_permeability(vclw, vsilt, phit)

    # Estimate rock types
    rock_flag = rock_typing(vclw, higher_is_better=False)

    # Update ZONES
    well_data = zone_flagging(well_data)
    zones = well_data.ZONES

    # Estimate reservoir summary
    ressum_df = calc_reservoir_summary(well_data.DEPTH, vclw, phit, swt, perm, zones, cutoffs=args['ressum_cutoffs'])

    # Update data in the project
    well_data['NPHI_HC'] = nphihc
    well_data['RHOB_HC'] = rhob_corr
    well_data['HC_FLAG'] = hc_flag
    well_data['VSAND'] = vsand
    well_data['VSILT'] = vsilt
    well_data['VCLD'] = vcld
    well_data['VCLW'] = vclw
    well_data['PHIT'] = phit
    well_data['PHIE'] = phie
    well_data['PHID'] = phid
    well_data['RW'] = rw
    well_data['B'] = b
    well_data['QV'] = qv
    well_data['M'] = m
    well_data['SWT'] = swt
    well_data['PERM'] = perm
    well_data['ROCK_FLAG'] = rock_flag  
    well_data['VHC'] = phit * (1 - swt)
    well_data['VSH_GR'] = vsh_gr

    # Save the results to the well
    well.update_data(well_data)
    well.update_ressum(ressum_df)
    project.save_well(well)


In [ ]:
# Plot individual results
fig = plotly_log(well_data, well.depth_uom)
fig.show()

In [ ]:
from quick_pp.qaqc import quick_compare

df = project.get_all_data()
compare_df, fig = quick_compare(df, return_fig=True)